<a href="https://colab.research.google.com/github/ACTP2002/EVIDENCE/blob/behavior_model/Behavior_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!pip install --no-cache-dir -U numpy scipy scikit-learn pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 29.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.0/35.0 MB 303.8 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.16.3
    Uninstalling scipy-1.16.3:
      Successfully uninstalled scipy-1.16.3


In [1]:
import numpy as np
import pandas as pd

# from sklearn.compose import ColumnTransformer
# from sklearn.preprocessing import OneHotEncoder, StandardScaler
# from sklearn.pipeline import Pipeline
# from sklearn.impute import SimpleImputer
from sklearn.ensemble import IsolationForest

In [2]:
data_path = "synthetic_data_2026-02-07.csv"

df = pd.read_csv(data_path)
df.head()

,customer_id,full_name,country,declared_income,income_currency,risk_rating,account_id,account_status,total_deposits_30d,total_withdrawals_30d,...,timestamp,type,amount,currency,status,channel,counterparty,risk_flags,is_anomaly,Unnamed: 22
0,CUST00000,User 0 Name,ZAF,79238,USD,high,ACC00000,active,69729.44,43932.0,...,2023-11-06T16:28:18,trade,982.54,USD,pending,crypto,CP4752,NaN,FALSE,NaN
1,CUST00000,User 0 Name,ZAF,79238,USD,high,ACC00000,active,69729.44,43932.0,...,2023-01-19T17:26:00,trade,992.60,USD,completed,bank_transfer,CP4656,NaN,FALSE,NaN
2,CUST00000,User 0 Name,ZAF,79238,USD,high,ACC00000,active,69729.44,43932.0,...,2023-05-28T03:54:45,deposit,811.80,USD,completed,bank_transfer,NaN,geographic_impossibility,TRUE,NaN
3,CUST00000,User 0 Name,ZAF,79238,USD,high,ACC00000,active,69729.44,43932.0,...,2023-04-06T17:04:40,trade,394.19,USD,failed,card,CP3925,NaN,FALSE,NaN
4,CUST00000,User 0 Name,ZAF,79238,USD,high,ACC00000,active,69729.44,43932.0,...,2023-02-26T23:38:21,trade,1017.67,USD,completed,bank_transfer,CP5664,NaN,FALSE,NaN


In [4]:
X = df.drop(columns=["is_anomaly", "Unnamed: 22"])
y = df["is_anomaly"]

print(X.head())
print(y.head())

  customer_id    full_name country  declared_income income_currency  \
0   CUST00000  User 0 Name     ZAF            79238             USD   
1   CUST00000  User 0 Name     ZAF            79238             USD   
2   CUST00000  User 0 Name     ZAF            79238             USD   
3   CUST00000  User 0 Name     ZAF            79238             USD   
4   CUST00000  User 0 Name     ZAF            79238             USD   

  risk_rating account_id account_status  total_deposits_30d  \
0        high   ACC00000         active            69729.44   
1        high   ACC00000         active            69729.44   
2        high   ACC00000         active            69729.44   
3        high   ACC00000         active            69729.44   
4        high   ACC00000         active            69729.44   

   total_withdrawals_30d  ...  deposit_to_income_ratio  transaction_id  \
0                43932.0  ...                     0.88     TXN00000000   
1                43932.0  ...                 

In [5]:
# Sort and key columns make it standardized lowercase
df = df.sort_values(["customer_id", "timestamp"]).reset_index(drop=True)

# Optional: normalize string columns
for c in ["country","income_currency","risk_rating","account_status","type","status","channel","counterparty","risk_flags","currency"]:
    if c in df.columns:
        df[c] = df[c].astype(str).str.strip().str.lower()

In [11]:
df.head()
len(df)

20000

Feature Engineering Process
- distinguish transaction type (1=d

In [16]:
def safe_div(a, b):
    b = np.where(b == 0, np.nan, b)
    return a / b

In [12]:
# Types of Transactions
df["is_deposit"] = (df["type"] == "deposit").astype(int)
df["is_withdrawal"] = (df["type"] == "withdrawal").astype(int)
df["is_transfer"] = (df["type"] == "transfer").astype(int)
df["is_trade"] = (df["type"] == "trade").astype(int)

In [15]:
type_cols = ["is_deposit", "is_withdrawal", "is_transfer", "is_trade"]
assert (df[["is_deposit","is_withdrawal","is_transfer","is_trade"]].sum(axis=1) == 1).all()

print(df["is_deposit"])

0        0
1        1
2        1
3        0
4        0
        ..
19995    0
19996    0
19997    1
19998    0
19999    0
Name: is_deposit, Length: 20000, dtype: int64


In [17]:
from scipy.stats import median_abs_deviation

# Amount in absolute
df["amount_abs"] = df["amount"].abs()

# Customer rolling stats (behavior baselines)
grp = df.groupby("customer_id", sort=False)

# Rolling windows (15 transactions minimum, 7 transactions at least to compute)
df["amt_roll_med_15"] = grp["amount_abs"].transform(lambda s: s.rolling(15, min_periods=7).median())
# Rolling MAD
df["amt_roll_mad_15"] = grp["amount_abs"].transform(lambda s: s.rolling(15, min_periods=7).apply(
    lambda x: median_abs_deviation(x, scale="normal"), raw=False
))

# Behavior deviation features
# Deviation from customer baseline
df["amt_dev_from_med"] = df["amount_abs"] - df["amt_roll_med_15"]
# Rolling robust z-score
df["amt_robust_z_roll"] = df["amt_dev_from_med"] / (df["amt_roll_mad_15"] + 1e-9)

In [19]:
# Ensure timestamp is datetime
df["timestamp"] = pd.to_datetime(df["timestamp"], errors="coerce")

# Previous timestamp per group
df["prev_ts"] = grp["timestamp"].shift(1)

# Gap between consecutive transactions (in seconds)
df["gap_seconds"] = (df["timestamp"] - df["prev_ts"]).dt.total_seconds()

# Fill first-transaction gaps
df["gap_seconds"] = df["gap_seconds"].fillna(df["gap_seconds"].median())

# Log-scaled time gap
df["gap_log"] = np.log1p(df["gap_seconds"])


In [20]:
# Risk flag cleanup

df["has_risk_flags"] = (
    df["risk_flags"].fillna("").astype(str).str.len() > 0
).astype(int)


In [24]:
df["is_new_counterparty_15"] = 0

for cid, idx in grp.groups.items():
    counterparties = df.loc[idx, "counterparty"].tolist()
    flags = []

    for i in range(len(counterparties)):
        if i == 0:
            flags.append(0)
        else:
            start = max(0, i - 30)
            history = counterparties[start:i]
            flags.append(int(counterparties[i] not in history))

    df.loc[idx, "is_new_counterparty_15"] = flags


In [25]:
# Global chronological split (simple)
cutoff = df["timestamp"].quantile(0.8)
train_df = df[df["timestamp"] <= cutoff].copy()
test_df  = df[df["timestamp"] > cutoff].copy()

print("Train:", train_df.shape, "Test:", test_df.shape)


Train: (16000, 37) Test: (4000, 37)


In [26]:
num_features = [
    "amount_abs", "gap_log",
    "total_deposits_30d", "total_withdrawals_30d",
    "average_transaction_amount", "deposit_to_income_ratio",
    "amt_robust_z_roll",
    "has_risk_flags", "is_new_counterparty_15",
]

cat_features = [
    "country", "currency", "channel", "status", "type",
    "account_status", "risk_rating"
]

# Ensure present
num_features = [c for c in num_features if c in df.columns]
cat_features = [c for c in cat_features if c in df.columns]


In [27]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import IsolationForest

preprocess = ColumnTransformer(
    transformers=[
        ("num", Pipeline(steps=[
            ("imputer", SimpleImputer(strategy="median")),
            ("scaler", RobustScaler())
        ]), num_features),
        ("cat", Pipeline(steps=[
            ("imputer", SimpleImputer(strategy="most_frequent")),
            ("oh", OneHotEncoder(handle_unknown="ignore", min_frequency=10))
        ]), cat_features),
    ],
    remainder="drop"
)

iso = IsolationForest(
    n_estimators=400,
    max_samples="auto",
    contamination="auto",
    random_state=42,
    n_jobs=-1
)

iso_pipe = Pipeline(steps=[("prep", preprocess), ("model", iso)])
iso_pipe.fit(train_df)

# Decision function: higher = more normal; lower = more anomalous
test_df["iforest_score_raw"] = iso_pipe.decision_function(test_df)
# Convert to anomaly score where higher = more anomalous
test_df["iforest_score"] = -test_df["iforest_score_raw"]


In [29]:
print(test_df["iforest_score"])

41      -0.029027
42       0.051347
43       0.033092
44      -0.009622
45      -0.026186
           ...   
19995   -0.075837
19996   -0.067865
19997    0.016983
19998   -0.066224
19999   -0.062575
Name: iforest_score, Length: 4000, dtype: float64
